In [17]:
import argparse
import os
import collections
from tqdm import tqdm 
from collections import defaultdict
import operator
from functools import reduce
from collections import Counter
import numpy as np
import string
import sys

In [56]:
#####################################################
##       Direcciones de archivos y argsparse       ##
#####################################################
#term either prefered (5) or as found in paper (4)
oger = '../temp/oger_annot_lemma.txt'
cluster_order = '../temp/abs.txt'
t = 5
typeof = 'preferred_'

In [57]:
#####################################################
##   Dic de pmid con palabras unicas por entidad   ##
#####################################################

pmids=defaultdict(dict)

with open (oger, 'r', encoding="utf-8") as file:
    for line in tqdm(file):
        pmid = line.split('\t')[0]
        if line[0].isdigit():
            #print(line)
            if pmid in pmids.keys():
                pmids[pmid][line.split('\t')[1]].add(line.split('\t')[t].strip('\n'))
            else:
                pmids[pmid]['biological_process']=set()
                pmids[pmid]['cellular_component']=set()
                pmids[pmid]['chemical']=set()
                pmids[pmid]['disease']=set()
                pmids[pmid]['gene/protein']=set()
                pmids[pmid]['molecular_function']=set()
                pmids[pmid]['organism']=set()
                pmids[pmid]['sequence']=set()
                pmids[pmid]['cell']=set()
                pmids[pmid]['cell_line']=set()
                pmids[pmid]['organ/tissue']=set()
                pmids[pmid]['clinical_drug']=set()
                pmids[pmid]['molecular_process']=set()
                pmids[pmid]['entity_type']=set()
                #pmids[pmid]=defaultdict(set)
                pmids[pmid][line.split('\t')[1]].add(line.split('\t')[t])


145290it [00:00, 175788.78it/s]


In [58]:
#####################################################
##  Dic de pmid con frecuencias unicas por entidad ##
#####################################################

pmids_frec=defaultdict(dict)

with open (oger, 'r', encoding="utf-8") as file:
    for line in tqdm(file):
        if line[0].isdigit():
            pmid = line.split('\t')[0]
            entidad = line.split('\t')[1]
            palabra = line.split('\t')[t]
            if pmid in pmids_frec.keys():
                if palabra in pmids_frec[pmid][entidad].keys():
                    pmids_frec[pmid][entidad][palabra]+=1
                else:
                    pmids_frec[pmid][line.split('\t')[1]][line.split('\t')[t]]=1
            else:
                pmids_frec[pmid]['biological_process']=defaultdict(dict)
                pmids_frec[pmid]['cellular_component']=defaultdict(dict)
                pmids_frec[pmid]['chemical']=defaultdict(dict)
                pmids_frec[pmid]['disease']=defaultdict(dict)
                pmids_frec[pmid]['gene/protein']=defaultdict(dict)
                pmids_frec[pmid]['molecular_function']=defaultdict(dict)
                pmids_frec[pmid]['organism']=defaultdict(dict)
                pmids_frec[pmid]['sequence']=defaultdict(dict)
                pmids_frec[pmid]['cell']=defaultdict(dict)
                pmids_frec[pmid]['cell_line']=defaultdict(dict)
                pmids_frec[pmid]['organ/tissue']=defaultdict(dict)
                pmids_frec[pmid]['clinical_drug']=defaultdict(dict)
                pmids_frec[pmid]['molecular_process']=defaultdict(dict)
                pmids_frec[pmid]['entity_type']=defaultdict(dict)
                pmids_frec[pmid][line.split('\t')[1]][line.split('\t')[t]]=1


145290it [00:01, 111566.89it/s]


In [59]:
#####################################################
##  txt de palabras de > a < frecuencia por pmid   ##
#####################################################    
for pmid in tqdm(pmids_frec):
    with open ('../temp/oger/by_pmid/'+typeof+pmid+'_frec.txt', 'w', encoding="utf-8") as file:
        for entidad in pmids_frec[pmid]:
            ent = sorted(pmids_frec[pmid][entidad].items(), key=lambda x: x[1], reverse=True)
            for tup in ent:
                file.write(pmid+'\t'+entidad+'\t'+tup[0]+'\t'+str(tup[1])+'\n')

100%|██████████| 1095/1095 [00:01<00:00, 693.33it/s]


In [60]:
#####################################################
##  dic de palabras de > a < frecuencia por clust  ##
#####################################################

with open (cluster_order, 'r', encoding="ISO-8859-1") as file:
    orden = [line.strip('\n').split('\t')[0] for line in file]
#print(orden)
clusters=defaultdict(dict)
for file in os.listdir('../results/clustering/'):
    #if file.startswith('k'):
    #print(file)
    with open ('../results/clustering/'+file, 'r',  encoding="ISO-8859-1") as f:
        cluster=file.strip('.txt')
        cont=0
        for line in f: 
            #print(line+'\n\n')
            key=line.split('\t')[2].strip('\n')
            if not line.startswith('dim'):
                if key in clusters[cluster].keys():
                    clusters[cluster][key].append(orden[cont])
                    cont+=1
                else:

                    clusters[cluster][key]=[]
                    clusters[cluster][key].append(orden[cont])
                    cont+=1

In [61]:
cluster_frecs=defaultdict(dict)
for met in clusters:
    cluster_frecs[met]=defaultdict(dict)
    for cluster in clusters[met]:
        cluster_frecs[met][cluster]=defaultdict(dict)
        for pmid in clusters[met][cluster]:
            
            for entidad in pmids_frec[pmid]:
                for palabra in pmids_frec[pmid][entidad]:
                    
                    if palabra in cluster_frecs[met][cluster][entidad]:
                        cluster_frecs[met][cluster][entidad][palabra][0]=pmids_frec[pmid][entidad][palabra]+cluster_frecs[met][cluster][entidad][palabra][0]
                    else: 
                        cluster_frecs[met][cluster][entidad][palabra]=[pmids_frec[pmid][entidad][palabra]]


In [62]:
#####################################################
##  txt de palabras de > a < frecuencia por pmid   ##
#####################################################    
for met in tqdm(cluster_frecs.keys()):
    with open ('../temp/oger/by_pmid/'+typeof+met+'_frec.txt', 'w', encoding='utf-8') as file:
        for cluster in cluster_frecs[met]:
            for entidad in cluster_frecs[met][cluster]:
                ent = sorted(cluster_frecs[met][cluster][entidad].items(), key=lambda x: x[1], reverse=True)
                for tup in ent:
                    file.write(cluster+'\t'+entidad+'\t'+tup[0]+'\t'+str(tup[1][0])+'\n')

100%|██████████| 1/1 [00:00<00:00, 11.29it/s]


In [63]:
#####################################################
##     # de clsuters en que aparece // tf-idf      ##
##################################################### 
entidades = ['cellular_component', 'chemical', 'gene/protein', 'organism', 'sequence', 'cell_line', 'organ/tissue', 'molecular_process', 'biological_process', 'clinical_drug', 'disease', 'molecular_function', 'entity_type', 'cell']


for entidad in entidades:
    for met in cluster_frecs:
        lista = []
        for cluster in cluster_frecs[met]:
            lista+=list(cluster_frecs[met][cluster][entidad].keys())
        dic = Counter(lista)
        for cluster in cluster_frecs[met]:
            for palabra in dic.keys():
                if palabra in cluster_frecs[met][cluster][entidad].keys():
                    cluster_frecs[met][cluster][entidad][palabra].append(dic[palabra])
        

In [64]:
#####################################################
##     # de clsuters en que aparece // tf/idf      ##
##################################################### 
# Frecuencia 1er posicion de list palabra
# # de apariciones 2da posicion
# frec * log (# clsuter totales / # de apariciones)

for met in tqdm(cluster_frecs):
    c_tot = len(cluster_frecs[met].keys())
    for cluster in cluster_frecs[met]:
        for entidad in cluster_frecs[met][cluster]:
            for palabra in cluster_frecs[met][cluster][entidad]:
                frec = cluster_frecs[met][cluster][entidad][palabra][0]
                apar = cluster_frecs[met][cluster][entidad][palabra][1]
                tfidf=frec*np.log10(c_tot/apar)
                cluster_frecs[met][cluster][entidad][palabra].append(tfidf)

100%|██████████| 1/1 [00:00<00:00,  6.36it/s]


In [65]:
#####################################################
##  txt de tfidf de > a < frecuencia por cluster   ##
#####################################################    
for met in tqdm(cluster_frecs):
    with open ('../temp/oger/by_cluster/'+typeof+met+'_5_tfidf.txt', 'w', encoding='utf-8') as file:
        for cluster in cluster_frecs[met]:
            for entidad in cluster_frecs[met][cluster]:
                ent = sorted(cluster_frecs[met][cluster][entidad].items(), key=lambda x: x[1][2], reverse=True)
                #print(ent)
                i = 0
                for pal in ent:
                    if i < 5:
                        #print(met, cluster, entidad, ent[i][0])
                        file.write(cluster+'\t'+entidad+'\t'+pal[0]+'\t'+str(pal[1][0])+'\t'+str(pal[1][1])+'\t'+str(pal[1][2])+'\n')
                        i+=1

100%|██████████| 1/1 [00:00<00:00, 19.03it/s]


In [66]:
#####################################################
##  txt de tfidf de > a < frecuencia por cluster   ##
#####################################################  
for met in tqdm(cluster_frecs):
    with open ('../temp/oger/by_cluster/'+typeof+met+'_10_tfidf.txt', 'w', encoding='utf-8') as file:
        for cluster in cluster_frecs[met]:
            for entidad in cluster_frecs[met][cluster]:
                ent = sorted(cluster_frecs[met][cluster][entidad].items(), key=lambda x: x[1][2], reverse=True)
                i = 0
                for pal in ent:
                    if i < 10:
                        file.write(cluster+'\t'+entidad+'\t'+pal[0]+'\t'+str(pal[1][0])+'\t'+str(pal[1][1])+'\t'+str(pal[1][2])+'\n')
                        i+=1

100%|██████████| 1/1 [00:00<00:00, 16.23it/s]
